In [1]:
import torch
from transformers import BertModel, BertConfig
from transformers import BertTokenizer

In [2]:
import sys
sys.path.append("../")

# create bert tokenizer and the bert pretrained model

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [4]:
text="Hello all! This is a test for the BERT model!"
text

'Hello all! This is a test for the BERT model!'

In [5]:
encoding = tokenizer(text, add_special_tokens = True, truncation = True, padding=True, return_attention_mask = True, return_tensors = "pt")

In [6]:
bert_config = BertConfig(output_hidden_states=False)

In [7]:
bert_model = BertModel.from_pretrained('bert-base-uncased', config=bert_config)

In [8]:
bert_model.config.hidden_size

768

# Create Vision Transformer

In [9]:
from dgs.blocks.vision.vit import Vit

In [10]:
image_size = 256
vit_model = Vit(image_size=image_size, vit_dim=bert_model.config.hidden_size, classifier_type=None) # use the output shape as the same size of bert

# Create the configurations for coattention_transformers and the output_transformer

In [11]:
# this is the parameters for the GeneralTransformerEncoderLayer which is same as nn.TransformerEncoderLayer
coattention_transformer_config = {
    "d_model": bert_model.config.hidden_size, 
    "nhead": 8, 
    "dim_feedforward": 1024, 
    "dropout": 0.1, 
    "activation": "gelu"
}

In [12]:
# which is same as nn.TransformerEncoderLayer
output_transformer_config = {
    "d_model": bert_model.config.hidden_size, 
    "nhead": 8, 
    "dim_feedforward": 1024, 
    "dropout": 0.1, 
    "activation": "gelu"
}

# Create the TILBERT model

In [13]:
from dgs.models.tilbert import TilBert

In [14]:
tilbert = TilBert(vit_model=vit_model, 
                  bert_model=bert_model, 
                  coattention_transformer_config=coattention_transformer_config, 
                  output_transformer_config=output_transformer_config,
                  num_of_combined_coattention_and_output_transformers=3,
                  classifier_type="token",
                  merge_mode="mul"
                 )

In [15]:
image_tensor = torch.randn(1,3,image_size, image_size)
image_tensor.shape

torch.Size([1, 3, 256, 256])

In [16]:
encoding["input_ids"].shape

torch.Size([1, 14])

In [17]:
multimodular_output = tilbert(image=image_tensor,
                              text_input_ids=encoding["input_ids"],
                              text_attention_mask=encoding["attention_mask"],
                              text_token_type_ids=encoding["token_type_ids"],
                             )

In [18]:
multimodular_output.shape

torch.Size([1, 768])